In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

from lobio.lob.limit_order import LimitOrder
from lobio.lob.order_book import OrderBook
from lobio.lob.price_level import PriceLevel, TraderId, Side

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
price_level = PriceLevel(LimitOrder(10, 1, Side.BUY, TraderId.MARKET))

price_level.add_limit_order(LimitOrder(10, 0.5, Side.BUY, TraderId.MARKET))

assert price_level == PriceLevel(LimitOrder(10, 1.5, Side.BUY, TraderId.MARKET))

price_level.add_limit_order(LimitOrder(10, 0.2, Side.BUY, TraderId.MM))

assert price_level.traders_order == [LimitOrder(10, 1.5, Side.BUY, TraderId.MARKET), LimitOrder(10, 0.2, Side.BUY, TraderId.MM)]
assert np.isclose(price_level.quote, 1.7, 1e-5)

price_level.add_limit_order(LimitOrder(10, 0.15, Side.BUY, TraderId.MARKET))

assert price_level.traders_order == [LimitOrder(10, 1.5, Side.BUY, TraderId.MARKET), 
                                    LimitOrder(10, 0.2, Side.BUY, TraderId.MM), 
                                    LimitOrder(10, 0.15, Side.BUY, TraderId.MARKET)]
assert np.isclose(price_level.quote, 1.85, 1e-5)

In [36]:
price_level = PriceLevel(LimitOrder(20, 50, Side.BUY, TraderId.MARKET))

remain_quote, data = price_level.execute_limit_order(20)

assert (remain_quote == 0.0)
assert (data == {TraderId.MARKET: 20})
assert (price_level == PriceLevel(LimitOrder(20, 30, Side.BUY, TraderId.MARKET)))

price_level.add_limit_order(LimitOrder(20, 5, Side.BUY, TraderId.MM))
remain_quote, data = price_level.execute_limit_order(31)

assert (remain_quote == 0.0)
assert (data == {TraderId.MARKET: 30, TraderId.MM: 1})
assert (price_level == PriceLevel(LimitOrder(20, 4, Side.BUY, TraderId.MM)))

remain_quote, data = price_level.execute_limit_order(100)

assert (remain_quote == 96)
assert (data == {TraderId.MM: 4})
assert (price_level.quote == 0 and price_level.traders_order == [])

In [41]:
init_lob = {"lastUpdateId": 1, "bids": [[90, 100]], "asks": [[100, 100]]}

diffs = [[2, [[90, 200]], [[100, 200]]], 
         [3, [[89.99, 99]], [[100.01, 300]]], 
         [4, [[94.99, 1], [89.99, 98]], [[95, 1], [100.01, 200]]],
         [5, [], [[95.01, 30], [95.5, 100], [100, 210]]],
         [6, [[87, 1], [90, 201], [94.99, 101]], []],
         [7, [[90, 51], [87, 0]], [[95.01, 1], [100, 0]]],
         [8, [[89.99, 0]], [[95, 0]]],
         [9, [[90.01, 51]], [[95, 100]]]]

In [42]:
ob = OrderBook.create_lob_init(init_lob)

ob.apply_historical_update(diffs[0])
assert ob == OrderBook.create_lob_init({"lastUpdateId": 2, 
                                       "bids": [[90, 200]], 
                                       "asks": [[100, 200]]})

ob.apply_historical_update(diffs[1])
assert ob == OrderBook.create_lob_init({"lastUpdateId": 3, 
                                       "bids": [[90, 200], [89.99, 99]], 
                                       "asks": [[100, 200], [100.01, 300]]})

ob.apply_historical_update(diffs[2])
assert ob == OrderBook.create_lob_init({"lastUpdateId": 4, 
                                       "bids": [[94.99, 1], [90, 200], [89.99, 98]], 
                                       "asks": [[95, 1], [100, 200], [100.01, 200]]})

ob.apply_historical_update(diffs[3])
assert ob == OrderBook.create_lob_init({"lastUpdateId": 5, 
                                       "bids": [[94.99, 1], [90, 200], [89.99, 98]], 
                                        "asks": [[95, 1], [95.01, 30], [95.5, 100], [100, 210], [100.01, 200]]})

ob.apply_historical_update(diffs[4])
assert ob == OrderBook.create_lob_init({"lastUpdateId": 6, 
                                       "bids": [[94.99, 101], [90, 201], [89.99, 98], [87, 1]], 
                                       "asks": [[95, 1], [95.01, 30], [95.5, 100], [100, 210], [100.01, 200]]})

ob.apply_historical_update(diffs[5])
assert ob == OrderBook.create_lob_init({"lastUpdateId": 7, 
                                       "bids": [[94.99, 101], [90, 51], [89.99, 98]],
                                       "asks": [[95, 1], [95.01, 1], [95.5, 100], [100.01, 200]]})

ob.apply_historical_update(diffs[6])
assert ob == OrderBook.create_lob_init({"lastUpdateId": 8, 
                                       "bids": [[94.99, 101], [90, 51]], 
                                       "asks": [[95.01, 1], [95.5, 100], [100.01, 200]]})

ob.apply_historical_update(diffs[7])
assert ob == OrderBook.create_lob_init({"lastUpdateId": 9, 
                                       "bids": [[94.99, 101], [90.01, 51], [90, 51]],
                                       "asks": [[95, 100], [95.01, 1], [95.5, 100], [100.01, 200]]})

In [48]:
init_lob = {"lastUpdateId": 1, "bids": [[48.37, 30], [48, 20]], "asks": [[50.45, 50], [50.47, 50]]}

In [49]:
ob = OrderBook.create_lob_init(init_lob)

data = ob.set_limit_order(LimitOrder(48.38, 10, Side.SELL, TraderId.MARKET))
assert ob == OrderBook.create_lob_init({"lastUpdateId": 2,
                                        "bids": [[48.37, 30], [48, 20]],
                                        "asks": [[48.38, 10], [50.45, 50], [50.47, 50]]})
assert data == {}

data = ob.set_limit_order(LimitOrder(48.38, 11, Side.BUY, TraderId.MARKET))
assert ob == OrderBook.create_lob_init({"lastUpdateId": 3,
                                        "bids": [[48.38, 1], [48.37, 30], [48, 20]],
                                        "asks": [[50.45, 50], [50.47, 50]]})
assert data == {TraderId.MARKET: [10, 10 * 48.38]}

data = ob.set_limit_order(LimitOrder(48.38, 1, Side.SELL, TraderId.MARKET))
assert ob == OrderBook.create_lob_init({"lastUpdateId": 4,
                                        "bids": [[48.37, 30], [48, 20]],
                                        "asks": [[50.45, 50], [50.47, 50]]})
assert data == {TraderId.MARKET: [1, 1 * 48.38]}

data = ob.set_limit_order(LimitOrder(50.45, 50, Side.BUY, TraderId.MARKET))
assert ob == OrderBook.create_lob_init({"lastUpdateId": 5,
                                        "bids": [[48.37, 30], [48, 20]],
                                        "asks": [[50.47, 50]]})
assert data == {TraderId.MARKET: [50, 50 * 50.45]}

data = ob.set_limit_order(LimitOrder(48, 49, Side.SELL, TraderId.MARKET))
assert ob == OrderBook.create_lob_init({"lastUpdateId": 6,
                                        "bids": [[48, 1]],
                                        "asks": [[50.47, 50]]})
assert data == {TraderId.MARKET: [49, 30 * 48.37 + 19 * 48]}

data = ob.set_limit_order(LimitOrder(50.47, 50, Side.BUY, TraderId.MARKET))
assert ob == OrderBook.create_lob_init({"lastUpdateId": 7,
                                        "bids": [[48, 1]],
                                        "asks": []})
assert data == {TraderId.MARKET: [50, 50 * 50.47]}

In [51]:
init_lob = {"lastUpdateId": 1, "bids": [], "asks": []}

In [52]:
ob = OrderBook.create_lob_init(init_lob)

data = ob.set_limit_order(LimitOrder(33, 50, Side.SELL, TraderId.MARKET))
assert ob == OrderBook.create_lob_init({"lastUpdateId": 2,
                                        "bids": [],
                                        "asks": [[33, 50]]})
assert data == {}

data = ob.set_limit_order(LimitOrder(34, 30, Side.BUY, TraderId.MM))
assert ob == OrderBook.create_lob_init({"lastUpdateId": 3,
                                        "bids": [],
                                        "asks": [[33, 20]]})
assert data == {TraderId.MARKET: [30, 30 * 33]}

data = ob.set_limit_order(LimitOrder(32.5, 40, Side.BUY, TraderId.MM))
ob_true = OrderBook.create_lob_init({"lastUpdateId": 4,
                                        "bids": [[32.5, 40]],
                                        "asks": [[33, 20]]})

ob_true.bids[0].traders_order[0].trader_id = TraderId.MM
assert ob == ob_true
assert data == {}

data = ob.set_limit_order(LimitOrder(32.5, 20, Side.SELL, TraderId.MARKET))
ob_true = OrderBook.create_lob_init({"lastUpdateId": 5,
                                        "bids": [[32.5, 20]],
                                        "asks": [[33, 20]]})
ob_true.bids[0].traders_order[0].trader_id = TraderId.MM
assert ob == ob_true
assert data == {TraderId.MM: [20, 20 * 32.5]}

data = ob.set_limit_order(LimitOrder(32.7, 100, Side.BUY, TraderId.MARKET))
ob_true = OrderBook.create_lob_init({"lastUpdateId": 6,
                                        "bids": [[32.7, 100], [32.5, 20]],
                                        "asks": [[33, 20]]})
ob_true.bids[1].traders_order[0].trader_id = TraderId.MM
assert ob == ob_true
assert data == {}

data = ob.set_limit_order(LimitOrder(33, 70, Side.BUY, TraderId.MM))
ob_true = OrderBook.create_lob_init({"lastUpdateId": 7,
                                        "bids": [[33, 50], [32.7, 100], [32.5, 20]],
                                        "asks": []})
ob_true.bids[0].traders_order[0].trader_id = TraderId.MM
ob_true.bids[2].traders_order[0].trader_id = TraderId.MM
assert ob == ob_true
assert data == {TraderId.MARKET: [20, 20 * 33]}